In [19]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import re

In [59]:
driver = webdriver.Chrome()

In [60]:
# let's start with democratie and citoyennete
# this will make us log in
# going to our page
driver.get('https://granddebat.fr/project/democratie-et-citoyennete-1/collect/participez-a-la-recherche-collective-de-solutions')

In [62]:
# logging in
button = driver.find_element_by_id('trash-link')
driver.execute_script("arguments[0].scrollIntoView(true)", button)
button.click()
driver.find_element_by_class_name('btn-success').click()

In [63]:
driver.find_element_by_id('username').send_keys('v.viers@lse.ac.uk')

In [64]:
driver.find_element_by_id('password').send_keys('elGranDebato')

In [65]:
driver.find_element_by_id('confirm-login').click()

In [ ]:
# clicking on right link after logging in
button = driver.find_element_by_id('trash-link')
driver.execute_script("arguments[0].scrollIntoView(true)", button)
button.click()

In [32]:
# getting all links to scrape and info aboute each link
# note that this script is not adapted to multiple page yet
# but it would just take an additional while loop 
# in which you'd make selenium click on the next button everytime
proposition_container = driver.find_element_by_class_name('proposal-preview-trash_list')
propositions = proposition_container.find_elements_by_class_name('block')

all_links = []
for proposition in propositions:
    new_link = {}
    header = proposition.find_element_by_class_name('proposal__title')
    new_link['title'] = header.find_element_by_tag_name('a').text
    new_link['link'] = header.find_element_by_tag_name('a').get_attribute('href')
    new_link['reason'] = proposition.find_element_by_class_name('proposal__trashed-reason').text
    new_link['author'] = proposition.find_element_by_class_name('proposal__author').text
    new_link['date'] = proposition.find_element_by_class_name('proposal__date').text
    # storing in list 
    all_links.append(new_link)

In [33]:
# turning list into dataframe
df_to_scrape = pd.DataFrame(all_links)

In [35]:
# function to scrape content of each page
def scraping_each_proposition(row): 
    driver.get(row['link'])
    main = driver.find_element_by_class_name('main')
    driver.implicitly_wait(3) # wait for page to set up
    question_blocks = main.find_elements_by_class_name('block')
    question_scraped = {}
    for question_block in question_blocks:
        try:
            text_question = question_block.find_element_by_tag_name('h3').text
            try:
                text_answer = question_block.find_element_by_tag_name('p').text
            except:    
                text_answer = 'NONE'
            question_scraped['"' + text_question + '"'] = text_answer
        except:
            continue
    question_scraped['link'] = row['link']
    # returning pd.Series object
    # it will have the link
    # and then each question will be column
    # and each answer will be the content of a cell
    # each proposition will be a row in our dataframe, when calling the function
    return pd.Series(question_scraped)

In [36]:
# calling function
here = df_to_scrape.apply(scraping_each_proposition, axis=1)

In [ ]:
# merging the dataframe with links to scrape and scraped pages
merged_df = here.merge(df_to_scrape, right_on = 'link', left_on = 'link')

In [ ]:
# saving as csv
merged_df.to_csv('democratie_et_citoyennete_corbeille.csv', index=False)

### Now taking it further by scraping all corbeilles

In [70]:
# creating list of all remaining corbeilles
corbeilles = ['https://granddebat.fr/projects/lorganisation-de-letat-et-des-services-publics/trashed', 'https://granddebat.fr/projects/la-transition-ecologique/trashed', 'https://granddebat.fr/projects/la-fiscalite-et-les-depenses-publiques/trashed']

In [ ]:
for corbeille in corbeilles:
    # we are already logged in
    driver.get(corbeille)
    button = driver.find_element_by_id('trash-link')
    driver.execute_script("arguments[0].scrollIntoView(true)", button)
    button.click()
    proposition_container = driver.find_element_by_class_name('proposal-preview-trash_list')
    propositions = proposition_container.find_elements_by_class_name('block')
    # doing as previously
    all_links = []
    for proposition in propositions:
        new_link = {}
        header = proposition.find_element_by_class_name('proposal__title')
        new_link['title'] = header.find_element_by_tag_name('a').text
        new_link['link'] = header.find_element_by_tag_name('a').get_attribute('href')
        new_link['reason'] = proposition.find_element_by_class_name('proposal__trashed-reason').text
        new_link['author'] = proposition.find_element_by_class_name('proposal__author').text
        new_link['date'] = proposition.find_element_by_class_name('proposal__date').text
        # storing in list 
        all_links.append(new_link)
    # our df of links to scrape
    df_to_scrape = pd.DataFrame(all_links)
    # actually scraping
    here = df_to_scrape.apply(scraping_each_proposition, axis=1)
    # merging
    merged_df = here.merge(df_to_scrape, right_on = 'link', left_on = 'link')
    # getting the right slug for our csv
    reg = re.compile(r'projects/(.*)/trashed')
    title_csv = str(reg.search(corbeille).group(1)).replace("'", "")
    # exporting
    merged_df.to_csv( title_csv + ".csv", index=False)